In [3]:
#pip install googletrans==4.0.0-rc1


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Obtaining dependency information for httpx==0.13.3 from https://files.pythonhosted.org/packages/54/b4/698b284c6aed4d7c2b4fe3ba5df1fcf6093612423797e76fbb24890dd22f/httpx-0.13.3-py3-none-any.whl.metadata
  Obtaining dependency information for hstspreload from https://files.pythonhosted.org/packages/25/2a/e9ca0cdf8bd33d61d8d237b19e62d5c2f2666e482ae3fd45e0a80b8dcfd0/hstspreload-2024.3.1-py3-none-any.whl.metadata
  Obtaining dependency information for chardet==3.* from https://files.pythonhosted.org/packages/bc/a9/01ffebfb562e4274b6487b4bb1ddec7ca55ec7510b22e4c51f14098443b8/chardet-3.0.4-py2.py3-none-any.whl.metadata
  Obtaining dependency information for idna==2.* from https://files.pythonhosted.org/packages/a2/38/928ddce2273eaa564f6f50de919327bf3a00f091b5baba8dfa9460f3a8a8/idna-2.10-py2.py3-none-any.whl.metadata
  Obtaining dependency information for rfc3986<2,>=1.3 from https://files.

In [1]:
import pandas as pd
import re
from urllib import request

import bs4
import lxml
from googletrans import Translator


## Année souverainte

In [5]:
def get_df_pays_anneeSouv():
  url_annee_souverainete="https://fr.wikipedia.org/wiki/Liste_de_pays_par_date_d%27acquisition_de_la_souverainet%C3%A9?fbclid=IwAR1Gg4tdNmv7M6Qj0pmVr1fg5aTsKKy2JTdqdNvqf6Oo0YuzOWtCKA5bsho"
  request_text= request.urlopen(url_annee_souverainete).read()

  page = bs4.BeautifulSoup(request_text, "lxml")

  # Recuperer toutes les tables
  tables=page.find_all('table')

  # Parourir chaque table pour en extraire le pays et l'année de souveraineté
  res_pays_anneeSouv=[]

  for j in range(6)  : #traiter les 6 premieres tables seulement (Afrique,Amérique,Asie,Europe,Océanie,Pays transcontinents)
    table=tables[j]

    rows=table.find_all('tr')
    for i in range(2, len(rows)):
      # Recuperer tous les j d'une ligne i
      row=[td.text.rstrip() for td in rows[i].find_all('td')]

      # Traiter le cas où pour un pays on a plusieurs dates de souverainete (prendre la plus recente)
      if(len(row)==4) and res_pays_anneeSouv != []: #attention il est important de specifier ==4 car il faut prendre le premier element d'une ligne qui commence à partir de l'attribut indice=3 (date souv (voir exp cameroun))
        annee_souverainete_brut=row[0][-4:] #recuperer l'année de la date de souveraineté la plus recente
        annee_souverainete_brut=re.search(r'^[^(]*', annee_souverainete_brut)
        annee_souverainete_brut=str(annee_souverainete_brut.group())
        annee_souverainete=annee_souverainete_brut.replace(" ","")
        if(annee_souverainete.isdigit()): #si elle peut etre convertie en type int (et non pas une sous chaine de caractère)
          res_pays_anneeSouv[-1][1]=int(annee_souverainete)

      elif(len(row)==7) :
        pays=row[0][1:]
        annee_souverainete_brut=row[3]
        if(re.search(r'\(', annee_souverainete_brut) !=None):
          annee_souverainete_brut=re.search(r'^[^(]*', annee_souverainete_brut) #traiter les cas comme croatie et danemark
          annee_souverainete_brut=str(annee_souverainete_brut.group())
          annee_souverainete_brut=annee_souverainete_brut[:-1] #supprimer l'espace qui precède la parenthèse
        annee_souverainete_brut=annee_souverainete_brut[-4:]
        annee_souverainete=annee_souverainete_brut.replace(" ","")
        if(annee_souverainete.isdigit()): #si elle peut etre convertie en type int (et non pas une sous chaine de caractère)
          res_pays_anneeSouv.append([pays,int(annee_souverainete)]) #ne garder que le nom du pays et l'annee de souverainete

  #traduire le nom des pays du francais a l'anglais
  translator = Translator()

  def traduire_nomPays(nomPays_fr):
      traduction = translator.translate(nomPays_fr, src='fr', dest='en')
      return traduction.text

  #remplacer les noms des pays de la liste
  res_pays_anneeSouv = [[traduire_nomPays(nomPays_fr), annee] for nomPays_fr, annee in res_pays_anneeSouv]

  df_pays_anneeSouv=pd.DataFrame(res_pays_anneeSouv, columns=["Pays", "Annee_souverainete"])

  return df_pays_anneeSouv

df_pays_anneeSouv=get_df_pays_anneeSouv()

In [6]:
df_pays_anneeSouv

,Pays,Annee_souverainete
0,South Africa,1910
1,Algeria,1962
2,Angola,1975
3,Benign,1960
4,Botswana,1885
...,...,...
191,Tuvalu,1978
192,Vanuatu,1980
193,Egypt,1953
194,Russia,1480


In [7]:
df_pays_anneeSouv[df_pays_anneeSouv['Pays']=='France']

,Pays,Annee_souverainete
148,France,843


In [8]:
df_pays_anneeSouv.to_csv("../datasets_scraped/df_pays_anneeSouv.csv", index=False)

## Prix nobels

In [2]:
url_prixNobel="https://en.m.wikipedia.org/wiki/List_of_Nobel_laureates_by_country?fbclid=IwAR0wZpgXVFycT23Csc-PAOgO8T_2tsTK4obr_LNVmwX0Wu4T6S6ZjNaXsYc"
request_text_prixNobel= request.urlopen(url_prixNobel).read()

page_prixNobel = bs4.BeautifulSoup(request_text_prixNobel, "lxml")


### Pays - nbPrix

In [3]:
table_prixNobel= page_prixNobel.find("table", {"class":"wikitable"})

rows=table_prixNobel.find_all("tr")
res_pays_nbPrix=[]
for i in range(1,len(rows)):
  pays_brut,nbPrix_brut=rows[i].find_all("td")
  if(pays_brut.find("a") !=None):
    pays=pays_brut.find("a").get("title")
  else : # traiter le cas potentiel ou la page d'un pays n'est plus referencée
    pays=pays_brut.get("title")
    
  if(nbPrix_brut.find("a") !=None):
    nbPrix=nbPrix_brut.find("a").text.rstrip()
  else :
    nbPrix=nbPrix_brut.text.rstrip()

  if(nbPrix.isdigit()):
    res_pays_nbPrix.append([pays,int(nbPrix)])
  else :
    nbPrix=str(re.search(r'\d+',nbPrix ).group())
    #nbPrix=nbPrix.split('-')[1]
    res_pays_nbPrix.append([pays,int(nbPrix)])
    

res_pays_nbPrix

[['United States', 411],
 ['United Kingdom', 137],
 ['Germany', 115],
 ['France', 75],
 ['Sweden', 34],
 ['Russia', 30],
 ['Japan', 29],
 ['Canada', 27],
 ['Switzerland', 25],
 ['Austria', 25],
 ['Netherlands', 22],
 ['Italy', 21],
 ['Poland', 18],
 ['Hungary', 15],
 ['Australia', 14],
 ['Denmark', 14],
 ['Norway', 14],
 ['Israel', 13],
 ['India', 10],
 ['Belgium', 11],
 ['Republic of Ireland', 11],
 ['South Africa', 11],
 ['China', 8],
 ['Spain', 8],
 ['Czech Republic', 6],
 ['Argentina', 5],
 ['Finland', 5],
 ['Egypt', 4],
 ['Pakistan', 2],
 ['Romania', 4],
 ['Ukraine', 4],
 ['Taiwan', 1],
 ['Belarus', 3],
 ['Croatia', 3],
 ['Lithuania', 3],
 ['Mexico', 3],
 ['New Zealand', 3],
 ['Bosnia and Herzegovina', 2],
 ['Chile', 2],
 ['Colombia', 2],
 ['East Timor', 2],
 ['Greece', 2],
 ['Guatemala', 2],
 ['Liberia', 2],
 ['Luxembourg', 2],
 ['Portugal', 2],
 ['Saint Lucia', 2],
 ['Tunisia', 2],
 ['Turkey', 2],
 ['Iran', 2],
 ['Bangladesh', 1],
 ['Bulgaria', 1],
 ['Algeria', 2],
 ['Costa Rica

In [4]:
df_pays_nbPrix=pd.DataFrame(res_pays_nbPrix, columns=["Pays", "Nb_prixNobel"])
df_pays_nbPrix

,Pays,Nb_prixNobel
0,United States,411
1,United Kingdom,137
2,Germany,115
3,France,75
4,Sweden,34
...,...,...
76,Trinidad and Tobago,1
77,Venezuela,1
78,Vietnam,1
79,Yemen,1


In [5]:
df_pays_nbPrix.to_csv("../datasets_scraped/df_pays_nbPrix.csv", index=False)

### Pays - CategoriePrix

In [6]:
section_categPrix_pays=page_prixNobel.find("section", {"id":"mf-section-2"})
#list_pays=[h3.find("span").get("id") for h3 in section_categPrix_pays.find_all("h3")]
list_pays=[h3.find("span", {"class":"mw-headline"}).text.rstrip() for h3 in section_categPrix_pays.find_all("h3")]
list_laureats=section_categPrix_pays.find_all("ol")


In [ ]:
# res=[]
# for indice_pays in range(len(list_laureats)):
#   list_laureats_pays_i_brut=list_laureats[indice_pays].find_all("li")
#   list_laureats_pays_i=[]
#   for laureats_pays_i in list_laureats_pays_i_brut:
#     if(laureats_pays_i.find("a") !=None):
#       nom_laureats_pays_i=laureats_pays_i.find("a").get("title")

#     else :
#       if(laureats_pays_i.text.rstrip()!=""):
#         nom_laureats_pays_i=str(re.search(r'^[^,]*',laureats_pays_i.text.rstrip()).group())
#       else :
#         continue #ne pas ajouter de '' dans la liste

#     list_laureats_pays_i.append(nom_laureats_pays_i)
#   res.append(list_laureats_pays_i)
# print(res)

In [7]:
res_laureat_categorie_annee=[]
ol_list_laureats_pays_i_brut=[]
next_sibling_of_ol=None
for indice_pays in range(len(list_laureats)):
  next_sibling_of_h3=list_laureats[indice_pays].find_next_sibling()

  if(next_sibling_of_h3 is not None and next_sibling_of_h3.name=='ol'): #traiter le cas où plusieurs ol se suivent entre deux balises de types h3
    next_sibling_of_ol=next_sibling_of_h3.find_next_sibling()
    ol_list_laureats_pays_i_brut.append(list_laureats[indice_pays].find_all("li"))
    ol_list_laureats_pays_i_brut.append(next_sibling_of_h3.find_all("li")) # sauvegarder au fur et a mesure tous les li des ol rencontrés
    continue

  elif(next_sibling_of_h3 is not None and next_sibling_of_h3.name=='h3')or (indice_pays==len(list_laureats)-1): #enregistrer les li contenus dans une (ou plusieurs) balise ol
      if(ol_list_laureats_pays_i_brut==[]):
        list_laureats_pays_i_brut=list_laureats[indice_pays].find_all("li")
        list_laureats_pays_i=[]
        for laureats_pays_i in list_laureats_pays_i_brut:
          if(laureats_pays_i.find("a") !=None):
            nom_laureats_pays_i=laureats_pays_i.find("a").get("title")
            txt_laureats_pays_i=str(laureats_pays_i.text.rstrip()).split(",")
            #nom_laureats_pays_i=txt_laureats_pays_i[0]
            if(len(txt_laureats_pays_i)>=3):
              categorie=txt_laureats_pays_i[-2][1:]
              annee=str(re.search(r'\d{4}',txt_laureats_pays_i[-1]).group())
            else :
              categorie=str(re.search(r'^[^(1-9)]*',txt_laureats_pays_i[-1]).group())
              annee=str(re.search(r'\d{4}',txt_laureats_pays_i[-1]).group())
          else :
            if(laureats_pays_i.text.rstrip()!=""):
              #nom_laureats_pays_i=str(re.search(r'^[^,]*',laureats_pays_i.text.rstrip()).group())
              txt_laureats_pays_i=str(laureats_pays_i.text.rstrip()).split(",")
              nom_laureats_pays_i=txt_laureats_pays_i[0]
              if(len(txt_laureats_pays_i)>=3):
                  categorie=txt_laureats_pays_i[-2][1:]
                  annee=str(re.search(r'\d{4}',txt_laureats_pays_i[-1]).group())
              else : #traiter le cas ou la séparation est mal formater (exp taiwan : Samuel C. C. Ting, Physics 1976* pas de virgule entre categorie et annee)
                categorie=str(re.search(r'^[^(1-9)]*',txt_laureats_pays_i[-1]).group())
                annee=str(re.search(r'\d{4}',txt_laureats_pays_i[-1]).group())
            else :
              continue #ne pas ajouter de '' dans la liste
          list_laureats_pays_i.append([nom_laureats_pays_i, categorie, annee])


      else :  #traiter le cas où plusieurs ol se suivent entre deux balises de types h3 (vider la liste dans la liste resultante)
        list_laureats_pays_i=[]
        for list_laureats_pays_i_brut in ol_list_laureats_pays_i_brut :
          for laureats_pays_i in list_laureats_pays_i_brut:
            if(laureats_pays_i.find("a") !=None):
              nom_laureats_pays_i=laureats_pays_i.find("a").get("title")
              txt_laureats_pays_i=str(laureats_pays_i.text.rstrip()).split(",")
              #nom_laureats_pays_i=txt_laureats_pays_i[0]
              if(len(txt_laureats_pays_i)>=3):
                  categorie=txt_laureats_pays_i[-2][1:]
                  annee=str(re.search(r'\d{4}',txt_laureats_pays_i[-1]).group())
              else : #traiter le cas ou la séparation est mal formater (exp taiwan : Samuel C. C. Ting, Physics 1976* pas de virgule entre categorie et annee)
                categorie=str(re.search(r'^[^(1-9)]*',txt_laureats_pays_i[-1]).group())
                annee=str(re.search(r'\d{4}',txt_laureats_pays_i[-1]).group())
            else :
              if(laureats_pays_i.text.rstrip()!=""):
                #nom_laureats_pays_i=str(re.search(r'^[^,]*',laureats_pays_i.text.rstrip()).group())
                txt_laureats_pays_i=str(laureats_pays_i.text.rstrip()).split(",")
                nom_laureats_pays_i=txt_laureats_pays_i[0]
                if(len(txt_laureats_pays_i)>=3):
                  categorie=txt_laureats_pays_i[-2][1:]
                  annee=str(re.search(r'\d{4}',txt_laureats_pays_i[-1]).group())
                else : #traiter le cas ou la séparation est mal formater (exp taiwan : Samuel C. C. Ting, Physics 1976* pas de virgule entre categorie et annee)
                  categorie=str(re.search(r'^[^(1-9)]*',txt_laureats_pays_i[-1]).group())
                  annee=str(re.search(r'\d{4}',txt_laureats_pays_i[-1]).group())
              else :
                continue #ne pas ajouter de '' dans la liste
            list_laureats_pays_i.append([nom_laureats_pays_i, categorie, annee])
        ol_list_laureats_pays_i_brut=[]

  res_laureat_categorie_annee.append(list_laureats_pays_i)
print(res_laureat_categorie_annee)

[[['César Milstein', 'Physiology or Medicine', '1984'], ['Adolfo Pérez Esquivel', 'Peace', '1980'], ['Luis Federico Leloir', 'Chemistry', '1970'], ['Bernardo Houssay', 'Physiology or Medicine', '1947'], ['Carlos Saavedra Lamas', 'Peace', '1936']], [['Ardem Patapoutian', 'Physiology or Medicine', '2021']], [['Brian Schmidt', 'Physics', '2011'], ['Elizabeth Blackburn', 'Physiology or Medicine', '2009'], ['Barry Marshall', 'Physiology or Medicine', '2005'], ['Robin Warren', 'Physiology or Medicine', '2005'], ['Peter C. Doherty', 'Physiology or Medicine', '1996'], ['Rolf Zinkernagel', 'Physiology or Medicine', '1996'], ['John Cornforth', 'Chemistry', '1975'], ['Patrick White', 'Literature', '1973'], ['Alexander Prokhorov', 'Physics', '1964'], ['John Eccles (neurophysiologist)', 'Physiology or Medicine', '1963'], ['Frank Macfarlane Burnet', 'Physiology or Medicine', '1960'], ['Howard Florey', 'Physiology or Medicine', '1945'], ['William Lawrence Bragg', 'Physics', '1915'], ['William Henry B

In [9]:
len(res_laureat_categorie_annee)

82

In [10]:
res_pays_listCateg=[]
for pays, laureat_categ_annee in zip(list_pays,res_laureat_categorie_annee):
  list_categ=[]#set()
  for i in range(len(laureat_categ_annee)):
    #list_categ.add(laureat_categ_annee[i][1])
    list_categ.append(laureat_categ_annee[i][1])

  res_pays_listCateg.append([pays, list_categ])
res_pays_listCateg

[['Argentina',
  ['Physiology or Medicine',
   'Peace',
   'Chemistry',
   'Physiology or Medicine',
   'Peace']],
 ['Armenia', ['Physiology or Medicine']],
 ['Australia',
  ['Physics',
   'Physiology or Medicine',
   'Physiology or Medicine',
   'Physiology or Medicine',
   'Physiology or Medicine',
   'Physiology or Medicine',
   'Chemistry',
   'Literature',
   'Physics',
   'Physiology or Medicine',
   'Physiology or Medicine',
   'Physiology or Medicine',
   'Physics',
   'Physics']],
 ['Austria',
  ['Physics',
   'Literature',
   'Chemistry',
   'Literature',
   'Physiology or Medicine',
   'Chemistry',
   'Economics',
   'Physiology or Medicine',
   'Physiology or Medicine',
   'Chemistry',
   'Physiology or Medicine',
   'Physiology or Medicine',
   'Physics',
   'Chemistry',
   'Physiology or Medicine',
   'Physics',
   'Physics',
   'Physiology or Medicine',
   'Physiology or Medicine',
   'Chemistry',
   'Peace',
   'Peace',
   'Chemistry',
   'Literature']],
 ['Bangladesh',

In [11]:
res_pays_categ=[[pays, categ] for pays, list_categ in res_pays_listCateg for categ in list_categ]
res_pays_categ

[['Argentina', 'Physiology or Medicine'],
 ['Argentina', 'Peace'],
 ['Argentina', 'Chemistry'],
 ['Argentina', 'Physiology or Medicine'],
 ['Argentina', 'Peace'],
 ['Armenia', 'Physiology or Medicine'],
 ['Australia', 'Physics'],
 ['Australia', 'Physiology or Medicine'],
 ['Australia', 'Physiology or Medicine'],
 ['Australia', 'Physiology or Medicine'],
 ['Australia', 'Physiology or Medicine'],
 ['Australia', 'Physiology or Medicine'],
 ['Australia', 'Chemistry'],
 ['Australia', 'Literature'],
 ['Australia', 'Physics'],
 ['Australia', 'Physiology or Medicine'],
 ['Australia', 'Physiology or Medicine'],
 ['Australia', 'Physiology or Medicine'],
 ['Australia', 'Physics'],
 ['Australia', 'Physics'],
 ['Austria', 'Physics'],
 ['Austria', 'Literature'],
 ['Austria', 'Chemistry'],
 ['Austria', 'Literature'],
 ['Austria', 'Physiology or Medicine'],
 ['Austria', 'Chemistry'],
 ['Austria', 'Economics'],
 ['Austria', 'Physiology or Medicine'],
 ['Austria', 'Physiology or Medicine'],
 ['Austria',

In [12]:
df_pays_CategoriePrixNobel=pd.DataFrame(res_pays_categ, columns=["Pays",'CategoriePrixNobel'])
df_pays_CategoriePrixNobel.head()

,Pays,CategoriePrixNobel
0,Argentina,Physiology or Medicine
1,Argentina,Peace
2,Argentina,Chemistry
3,Argentina,Physiology or Medicine
4,Argentina,Peace


In [13]:
df_pays_CategoriePrixNobel.to_csv("../datasets_scraped/df_pays_CategoriePrixNobel.csv", index=False)

In [19]:
df_pays=df_pays_CategoriePrixNobel[df_pays_CategoriePrixNobel.iloc[:,0]=="China (People's Republic of China)"]
df_pays

,Pays,CategoriePrixNobel
92,China (People's Republic of China),Physiology or Medicine
93,China (People's Republic of China),Literature
94,China (People's Republic of China),Peace
95,China (People's Republic of China),Physics
96,China (People's Republic of China),Literature
97,China (People's Republic of China),Physics
98,China (People's Republic of China),Physics
99,China (People's Republic of China),Physics
